In [0]:
! wget https://www.dropbox.com/s/qmj7tefad2vysbb/Archive.zip?dl=0 -O datazip.zip 
! unzip datazip.zip -d data

In [0]:
! wget https://drive.google.com/open?id=1e3-EVn3AIgJNZA8BmYyScV2qHMibnK1e -O itdraw.npy

In [0]:
!pip install mne
!pip install EMD-signal
!pip install nolds
!pip install matlab

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mne
import h5py
import pywt
import random
import os
import nolds
import scipy.io as sio
import matlab.engine
from scipy.stats import kurtosis, skew
from PyEMD import EMD, EEMD, Visualisation
from scipy import stats
from scipy.signal import butter, lfilter, freqz, spectrogram, periodogram, stft, find_peaks
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from imblearn.over_sampling import ADASYN, SMOTE

fs = 256
szstart2 = 579840
szend2 = 587520

def butter_lowpass(data, cutoff, fs, order=5):
    ny = fs/2
    b, a = butter(order, cutoff/ny, btype='lowpass', analog=False)
    y = lfilter(b,a,data)
    return y

def butter_bandpass(data, low, high, fs, order=5):
    ny = fs/2
    b, a = butter(order, [low/ny, high/ny], btype='bandpass')
    y = lfilter(b,a,data)
    return y

def butter_bandstop(data, low, high, fs, order=5):
    ny = fs/2
    b, a = butter(order, [low/ny, high/ny], btype='bandstop')
    y = lfilter(b, a, data)
    return y

def kagdat(files,segnum,types):
	f = sio.loadmat(files)
	if types == 'pre':
		struct = f[('preictal_segment_' + str(segnum))]
	else:
		struct = f[('interictal_segment_' + str(segnum))]
	val = struct[0,0]
	signal = val['data']
	signal = butter_bandpass(signal,0.5,127,fs)
	signal = butter_bandstop(signal,47,53,fs)
	signal = butter_bandstop(signal,97,103,fs)
	return signal

def segmentkag(xtrain,xtest,ytrain,ytest,species,seconds):
	atrain = []
	atest = []
	aytrain = []
	aytest = []

	if species == 'dog':
		frez = 400
	else:
		frez = 5000
	for s in range(0,len(xtrain)):
		seg = xtrain[s]
		i = 0
		while (i+1)*frez*seconds < len(seg[0]):
			atrain += [seg[:,(i*frez*seconds):((i+1)*frez*seconds)]]
			if ytrain[s] == 1:
				aytrain += [1]
			else:
				aytrain += [0]
			i += 1
	for s in range(0,len(xtest)):
		seg = xtest[s]
		i = 0
		while (i+1)*frez*seconds < len(seg[0]):
			atest += [seg[:,(i*frez*seconds):((i+1)*frez*seconds)]]
			if ytest[s] == 1:
				aytest += [1]
			else:
				aytest += [0]
			i += 1
	return atrain,atest,aytrain,aytest

def getkaggle(dire):
	path = os.path.join('kaggle',dire)
	files = sorted(os.listdir(path))
	preictal = []
	interictal = []
	for y in files:
		if y.find('.DS_Store') != -1:
			pass
		else:
			x = os.path.join(dire,y)
			x = os.path.join('kaggle',x)
			if x.find('preictal') != -1:
				preictal += [x]
			else:
				interictal  += [x]
	#print(preictal[0])
	preictal = sorted(preictal)
	interictal = sorted(interictal)

	precut = (len(preictal)*4) // 5
	intcut = (len(interictal)*4) // 5

	train = []
	test = []
	ytrain = []
	ytest = []

	for i in range(0,len(preictal),1):
		if i < precut:
			train += [kagdat(preictal[i],i+1,'pre')]
			ytrain += [1]
		else:
			test += [kagdat(preictal[i],i+1,'pre')]
			ytest += [1]

	for i in range(0,len(interictal),1):
		if i < intcut:
			train += [kagdat(interictal[i],i+1,'int')]
			ytrain += [0]
		else:
			test += [kagdat(interictal[i],i+1,'int')]
			ytest += [0]
			
	return train,test,ytrain,ytest

def bonn(files):
	x = np.genfromtxt(files, dtype=None)
	x = butter_bandpass(x,0.5,127,173.61)
	x = butter_bandstop(x,47,53,173.61)
	x = butter_bandstop(x,97,103,173.61)
	return x

def getbonn(dire):
	path = os.path.join('Bonn',dire)
	files = sorted(os.listdir(path))
	data = []
	for y in files:
		if y.find('.DS_Store') != -1:
			pass
		else:
			x = os.path.join(dire,y)
			x = os.path.join('Bonn',x)
			data += [x]

	#print(preictal[0])

	datcut = (len(data)*4) // 5

	train = []
	test = []
	ytrain = []
	ytest = []

	for i in range(0,len(data),1):
		if i < precut:
			train += [bonn(data[i])]
			if dire == 'ictal':
				ytrain += [1]
			else:
				ytrain += [0]
		else:
			test += [bonn(preictal[i],i+1,'pre')]
			if dire == 'ictal':
				ytest += [1]
			else:
				ytest += [0]
			
	return train,test,ytrain,ytest
def raw(files):
	#Opens matlab raw data file and returns raw signal array and label array
	f = h5py.File(files,'r')
	signal = f['signal'].value
	'''Apply 0.5-140 Hz Bandpass Filter and 47-53 + 97-103 Hz Bandstop Filter'''
	signal = butter_bandpass(signal,0.5,127,fs)
	signal = butter_bandstop(signal,47,53,fs)
	signal = butter_bandstop(signal,97,103,fs)

	'''Get Seizure Start'''
	test = f['sample_scsz_onset']
	try:
		st = test[0][0]
		obj = f[st]
		szstart = float(obj.value)
		szstart = round(szstart)
	except:
		szstart = -1

	'''Get Seizure End'''
	try:
		tes = f['sample_scsz_end']
		s = tes[0][0]
		ob = f[s]
		szend = float(ob.value)
		szend = round(szend)
	except:
		szend = -1
	'''Create Label Array'''
	labels = np.zeros(len(signal[1]))
	if szstart != -1:
		labels[szstart:szend] = 1

	return signal, labels, szstart, szend

def getfreqbands(signal,fs):
	'''Bands from O'Leary'''
	delta = butter_bandpass(signal,0.5,4,fs)
	theta = butter_bandpass(signal,4,8,fs)
	alpha = butter_bandpass(signal,8,13,fs)
	beta = butter_bandpass(signal,13,30,fs)
	gamma = butter_bandpass(signal,30,60,fs)
	other = butter_bandpass(signal,60,127,fs)

	return delta,theta,alpha,beta,gamma,other

def makeictal(signal,szstart,szend):
	'''Use Overlapped Ictal Sampling to generate more ictal samples'''
	ictal_batches = []
	ictal_segments = []
	if (szstart - (fs*32)) < 0:
		start = 0
	else:
		start = szstart - (fs*32)

	if (szend + (fs*32)) > len(signal[0]):
		end = len(signal)
	else:
		end = szend + (fs*32)

	temp = signal[:,start:end]

	i = 0

	while True:
		if ((i+64)*fs) > len(temp[0]):
			break
		ictal_batches += [temp[:,i*fs:(i+64)*fs]]
		i += 1

	'''Turn Ictal batches into 1s segments'''
	for batch in ictal_batches:
		i = 0
		while True:
			if ((i+1)*fs) > len(batch[0]):
				break
			ictal_segments += [batch[:,i*fs:(i+1)*fs]]
			i += 1
	return ictal_segments

def shuffle(train,label):
	c = list(zip(train,label))
	random.shuffle(c)
	x_train, y_train = zip(*c)
	x_train = list(x_train)
	y_train = list(y_train)

	y_train = np.asarray(y_train)
	x_train = np.asarray(x_train)
	return x_train,y_train

def segment_data(sig,label,shuffle,seconds):

	fs = 256
	#print(sig.shape)
	#print(label.shape)
	data = np.concatenate((sig,[label]), axis=0)
	x_temp = []
	y_temp = []
	i = 0
	while True:
		if ((i+1)*fs*seconds) > len(data[0]):
			break
		x_temp += [data[0:16,i*fs*seconds:(i+1)*fs*seconds]]
		if np.mean(data[16,i*fs*seconds:(i+1)*fs*seconds]) > 0.1:
			y_temp += [1]
		else:
			y_temp += [0]
		i += 1

	if shuffle == True:
		c = list(zip(x_temp,y_temp))
		random.shuffle(c)
		x_temp, y_temp = zip(*c)
		x_temp = list(x_temp)
		y_temp = list(y_temp)

	cut = (len(x_temp)*4) // 5

	x_train = x_temp[0:cut]
	x_test = x_temp[cut:len(x_temp)]

	y_train = y_temp[0:cut]
	y_test = y_temp[cut:len(y_temp)]
	return x_train, y_train, x_test, y_test

def segmentx(sig,seconds):
	fs = 256
	x_temp = []
	i = 0
	while True:
		if ((i+1)*fs*seconds) > len(sig[0]):
			break
		x_temp += [sig[0:16,i*fs*seconds:(i+1)*fs*seconds]]
	cut = (len(x_temp)*4) // 5

	x_train = x_temp[0:cut]
	x_test = x_temp[cut:len(x_temp)]

	return x_train,x_test
def wavelet_raw(sig):
	'''Apply Wavlet Transform and Get Co-efficients'''
	cA5, cD5, cD4, cD3, cD2, cD1 = pywt.wavedec(sig, 'db4', level=5)
	coeff = [cA5,cD5,cD4,cD3,cD2,cD1]
	accum = np.concatenate((cA5,cD5,cD4,cD3,cD2,cD1),axis=1)

	#accum = accum.flatten()
	'''accum shape (16,288)'''
	#print(accum.shape)
	return accum

def wavepack(sig):
	#print(pywt.wavelist())
	print('hi')
	wp = pywt.WaveletPacket2D(data=sig,wavelet='db1',maxlevel=3)
	print('hi')
	a = wp['aa'].data
	print(a.shape)
	return a
def spectrograms(sig,fs,windowlength):
	accum = []
	for i in range(0,len(sig)):
		f,t,tempsig = spectrogram(sig[i],fs=fs,nperseg=windowlength)

		'''
		plt.pcolormesh(t,f,tempsig)
		plt.title('Power Spectogram')
		plt.xlabel('Time (s)')
		plt.ylabel('Frequency (Hz)')
		plt.show()
		'''
		#print(tempsig.shape)
		accum += [tempsig]
	#print(accum.shape)
	return accum

def stfts(sig,fs,windowlength):
	#accum = []
	
	f,t,tempsig = stft(sig,fs=fs,nperseg=windowlength)
		#plt.pcolormesh(t,f,np.abs(tempsig))
		#plt.show()
		#print(tempsig.shape)
		#accum += [tempsig]
	#print(accum.shape)
	return tempsig

def EMDs(signal):
	emd = EMD()
	emd.emd(signal)
	imfs, res = emd.get_imfs_and_residue()

	# In general:
	#components = EEMD()(S)
	#imfs, res = components[:-1], components[-1]
	'''
	t = np.arange(0,3,0.0025)
	vis = Visualisation()
	vis.plot_imfs(imfs=imfs, residue=res, t=t, include_residue=True)
	vis.plot_instant_freq(t, imfs=imfs)
	vis.show()
	'''
	#print(imfs)
	print(imfs.shape)

	return imfs

def itd_base_ext(sig):
	x = np.transpose(sig)
	t = np.arange(len(x))
	alpha = 0.5
	idx_max,val_max = find_peaks(x,height=0)
	idx_min,val_min = find_peaks(np.negative(x),height=0)
	idx_cb = np.union1d(idx_max,idx_min)
	val_min = np.negative(val_min['peak_heights'])
	#val_min.reshape((-1,1))
	'''
	if min(idx_max) < min(idx_min):
		val_min = np.concatenate((val_min[0],val_min),axis=0)
		idx_min = np.concatenate(idx_min[0],idx_min)
	elif min(idx_max) > min(idx_min):
		val_max = val_max['peak_heights']
		#val_max.reshape((-1,1))
		val_max = np.concatenate((val_max[0],val_max))
		idx_min = np.concatenate(idx_min[0],idx_min)
	'''
	Max_line= np.interp(idx_max,val_max,t)
	Min_line= np.interp(idx_min,val_min,t)
	Lk1=alpha*Max_line[idx_min]+val_min*(1-alpha)
	Lk2=alpha*Min_line[idx_max]+val_max*(1-alpha)
	Lk1 = np.concatenate(idx_min,Lk1)
	Lk2 = np.concatenate(idx_max,Lk2)
	Lk = np.concatenate(Lk1,Lk2)
	inds = np.argsort(Lk[:,0])
	Lk_col_2 = np.sort(Lk[:,0])
	Lk_sorted = Lk[Lk_col_2,:]
	Lk = Lk_sorted[2:len(Lk)-1,:]
	Lk = np.concatenate([1,Lk[0,1]], Lk,[len(x),Lk(len(Lk) - 1, 2)])
	idx_xk = np.concatenate(1,idx_cb,len(x))
	L = np.zeros(1,len(x))
	for i in range(0,len(idx_xk)-1):
		for j in range(idx_xk(i),idx_xk(i+1)):
			kig = (Lk(i+2,2) - Lk(i,2)) / (x(idx_xk[i+1]) - x(idx_xk[i]))
			L[j] = Lk[i,2] + kig*(x[j]-x[idx_xk[i]])
	return x - L

def STOP(raw,count,N_max,squaresum):
	stop = 0
	if count > N_max:
		stop = 1
		return stop
	elif squaresum <= 0.01*squaresum:
		stop = 1
		return stop
	idx1,pk1 = find_peaks(raw,height=0)
	idx2,pk2 = find_peaks(np.negative(raw),height=0)
	pks = np.union1d(pk1['peak_heights'],pk2['peak_heights'])
	if len(pks) <= 7:
		stop = 1
	return stop

def IDT(signal):
	'''
	raw = signal
	N_max = 10
	sqauresum = sum(np.square(signal))
	count = 0
	while True:
		count += 1
		L1,H1 = itd_base_ext(raw)
		if count == 1:
			final = H1
		else:
			final = np.append(final,[H1],axis=0)
		STOP = stop_iter(raw,count,N_max,sqauresum)
		if STOP:
			final = np.append(final,[L1])
			break
	return final
	'''
	sio.savemat('sigt.mat', dict(sig=signal))
	eng = matlab.engine.start_matlab()
	a = eng.load('sigt.mat')
	out = eng.ITDmatlab(a['sig'])
	eng.quit()
	b = np.asarray(out)
	return b

def IDTraw(signal):
	accum = []
	#print(signal.shape)
	error = False
	for i in range(0,len(signal)):
		try:
			a = IDT(signal[i,:])
			accum += [a[0:5,:]]
		except Exception as e:
			error = True
		#print(a.shape)
	if error == True:
		return -1
	else:
		return np.asarray(accum)

def LLE(signal):
	exp = nolds.lyap_r(signal)
	return exp
def cordim(signal):
	cor = nolds.corr_dim(signal,1)
	return cor
def dfas(signal):
	return nolds.dfa(signal)
def ApEn(U, m, r):
	#use m = 2 and r = 0.2*standard deviation
    def _maxdist(x_i, x_j):
        return max([abs(ua - va) for ua, va in zip(x_i, x_j)])

    def _phi(m):
        x = [[U[j] for j in range(i, i + m - 1 + 1)] for i in range(N - m + 1)]
        C = [len([1 for x_j in x if _maxdist(x_i, x_j) <= r]) / (N - m + 1.0) for x_i in x]
        return (N - m + 1.0)**(-1) * sum(np.log(C))

    N = len(U)

    return abs(_phi(m + 1) - _phi(m))

def feature_extraction(sig):
	'''Get Statistical features from wavelet transform'''

	'''Apply Wavlet Transform and Get Co-efficients'''
	cA5, cD5, cD4, cD3, cD2, cD1 = pywt.wavedec(sig, 'db4', level=5)
	coeff = [cA5,cD5,cD4,cD3,cD2,cD1]

	'''Mean of Wavelet Co-efficients'''
	D1avg = []
	D2avg = []
	D3avg = []
	D4avg = []
	D5avg = []
	A5avg = []

	averages = [A5avg, D5avg, D4avg, D3avg, D2avg, D1avg]

	for i in range(0,len(averages),1):
		for x in coeff[i]:
			abso = np.absolute(x)
			mean = np.mean(abso)
			averages[i] += [mean]
	averages = np.asarray(averages)
	#PCA = PCA(n_components = 6)
	#PCA.fit(averages)


	'''Standard Deviation of Co-efficients'''
	D1sd = []
	D2sd = []
	D3sd = []
	D4sd = []
	D5sd = []
	A5sd = []

	deviations = [A5sd, D5sd, D4sd, D3sd, D2sd, D1sd]
	for i in range(0,len(deviations),1):
		for x in coeff[i]:
			sds = np.std(x)
			deviations[i] += [sds]
	deviations = np.asarray(deviations)
	#pcadev = PCA.fit(deviations)

	'''Average Power of Co-efficients'''
	D1pow = []
	D2pow = []
	D3pow = []
	D4pow = []
	D5pow = []
	A5pow = []

	powers = [A5pow, D5pow, D4pow, D3pow, D2pow, D1pow]
	for i in range(0,len(powers),1):
		for x in coeff[i]:
			avg_power = np.mean(np.random.power(np.abs(x)))
			powers[i] += [avg_power]
	powers = np.asarray(powers)
	#pcapow = PCA.fit(powers)
	'''Approximate Entropy of Co-efficients'''
	D1apen = []
	D2apen = []
	D3apen = []
	D4apen = []
	D5apen = []
	A5apen = []
	apens = [A5apen, D5apen, D4apen, D3apen, D2apen, D1apen]
	for i in range(0,len(powers),1):
		for x in coeff[i]:
			aproxen = ApEn(x,2,2)
			apens[i] += [aproxen]
	apens = np.asarray(apens)

	'''Largest Lypuanov Exponent of Co-efficients'''
	
	D1LLE = []
	D2LLE = []
	D3LLE = []
	D4LLE = []
	D5LLE = []
	A5LLE = []
	LLEs = [A5LLE, D5LLE, D4LLE, D3LLE, D2LLE, D1LLE]
	for i in range(0,len(powers),1):
		for x in coeff[i]:
			aproxen = LLE(x)
			LLEs[i] += [aproxen]
	LLEs = np.asarray(LLEs)

	'''DFA'''
	D1A = []
	D2A = []
	D3A = []
	D4A = []
	D5A = []
	A5A = []
	dfas = [A5A, D5A, D4A, D3A, D2A, D1A]
	for i in range(0,len(powers),1):
		for x in coeff[i]:
			aproxen = dfas(x)
			dfas[i] += [aproxen]
	dfas = np.asarray(dfas)

	'''Skewness'''
	D1S = []
	D2S = []
	D3S = []
	D4S = []
	D5S = []
	A5S = []
	skews = [A5S, D5S, D4S, D3S, D2S, D1S]
	for i in range(0,len(powers),1):
		for x in coeff[i]:
			aproxen = skew(x)
			skews[i] += [aproxen]
	skews = np.asarray(skews)

	'''Kurtosis'''
	D1K = []
	D2K = []
	D3K = []
	D4K = []
	D5K = []
	A5K = []
	kurts = [A5K, D5K, D4K, D3K, D2K, D1K]
	for i in range(0,len(powers),1):
		for x in coeff[i]:
			aproxen = kurtosis(x)
			kurts[i] += [aproxen]
	kurts = np.asarray(kurts)


	features = np.concatenate((averages, deviations, powers, apens, LLEs, dfas, skews, kurts), axis=0)
	#print(features.shape)
	return features

def Adasyn(x,y):
	sm = ADASYN(sampling_strategy=0.2)
	'''
	a = x.tolist()
	dim1 = len(a[0])
	dim2 = len(a[0][0])
	for i in range(0,len(a),1):
		accum = []
		for s in a[i]:
			accum += s
		a[i] = accum

	a = np.array(a)
	'''
	#print(a.shape)
	a, b = sm.fit_sample(x,y)
	c = list(zip(a,b))
	random.shuffle(c)
	a, b = zip(*c)
	a = list(a)
	b = list(b)
	#a = a.tolist()
	dim0 = len(a)
	a = np.array(a)
	#a = a.reshape((dim0,dim1,dim2))
	b = np.array(b)
	return (a, b)

if __name__ == "__main__":
	signal, labels, szstart, szend = raw('EDMSE_pat_FR_1096_005.mat')

	labels[szstart2:szend2] = 1
	xraw_train, y_train, xraw_test, y_test = segment_data(signal,labels,True)

	x_train = []
	x_test = []

	for x in xraw_train:	
		averages, deviations, powers = feature_extraction(x)
		x_train += [np.concatenate((averages, deviations), axis=0)]

	for t in xraw_test:
		averages, deviations, powers = feature_extraction(x)
		x_test += [np.concatenate((averages, deviations), axis=0)]

	ictal_segments = makeictal(signal,szstart,szend)

	dat = pd.read_csv('data/samples.csv')
	dat = dat.fillna(0)
	#ndat = np.delete(numpydat,0,axis=1)
	#numpydat = dat.values.transpose()
	#dattag = dat.keys()


	'''Set up Raw Data'''
	sfreq = 256
	ch_types = ['eeg', 'eeg', 'eeg', 'eeg']
	info = mne.create_info(ch_names=list(dat.keys()[1:17]), sfreq=sfreq, ch_types='eeg')
	raw = mne.io.RawArray(signal, info)


	'''Maybe Use: Ratio of the absolute mean values of adjacent subbands'''

	'''Graph Raw Data'''
	scalings = {'eeg': 200}
	raw.plot(n_channels=23, scalings=scalings, title='Data from arrays',
	         show=True, block=True)
	'''
	plt.plot(cD5[0])
	plt.plot(cD4[0])
	plt.plot(cD3[0])
	plt.plot(cD2[0])

	#plt.plot(cA5[0])
	plt.show()
	#plt.plot(dat["time"],dat["TP9"])
	#plt.show()
	#print(dat)"""
	'''


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
import sys
import os
import random
import umap
import numpy as np
import nolds
from random import sample
from numpy import inf
from scipy.stats import kurtosis, skew
from random import randint
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Conv1D, MaxPooling1D, Input, Lambda, BatchNormalization, Conv2D, MaxPooling2D, UpSampling1D
from keras.utils import np_utils, Sequence, to_categorical
from sklearn.metrics import f1_score, confusion_matrix
from keras.layers.advanced_activations import LeakyReLU
from keras.initializers import Zeros, TruncatedNormal,Constant
from sklearn.utils import class_weight
from sklearn import svm
from sklearn.decomposition import PCA, FastICA
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.manifold import Isomap
def mean(x):
	return (sum(x)/(len(x)))

def Model(nntype):
	winit = TruncatedNormal(mean=0,stddev=0.05)
	binit = Constant(value=0)
	if nntype == 'cnn':
		model = Sequential()
		model.add(Conv2D(16, kernel_size=3, padding='same',input_shape=(16,4,768)))
		model.add(Activation('relu'))
		#model.add(MaxPooling2D(pool_size=2, padding='same'))
		model.add(Dropout(0.7))
		model.add(Conv2D(32, kernel_size=3, padding='same'))
		model.add(Activation('relu'))
		model.add(MaxPooling2D(pool_size=2, padding='same'))
		model.add(Dropout(0.7))
		model.add(Conv2D(32, kernel_size=3, padding='same'))
		model.add(Activation('relu'))
		#model.add(MaxPooling2D(pool_size=2, padding='same'))
		model.add(Dropout(0.7))

		model.add(Conv2D(64, kernel_size=3, padding='same'))
		model.add(Activation('relu'))
		model.add(MaxPooling2D(pool_size=2, padding='same'))
		model.add(Dropout(0.7))

		model.add(Flatten())
		model.add(Dense(64))
		model.add(Activation('relu'))
		model.add(Dropout(0.7))
		model.add(Dense(16))
		model.add(Activation('relu'))
		model.add(Dropout(0.7))
		model.add(Dense(1, activation ='sigmoid'))
		return model
	if nntype == 'fc':
		model = Sequential()
		model.add(Dense(10,input_dim=1,kernel_initializer=winit,
                bias_initializer=binit))
		model.add(Activation(activation='relu'))
		
		model.add(Dropout(0.5))
		model.add(Dense(10,kernel_initializer=winit,
                bias_initializer=binit))
		model.add(Activation(activation='relu'))
		
		model.add(Dropout(0.7))
		model.add(Dense(1, activation='sigmoid',kernel_initializer=winit,
                bias_initializer=binit))
		return model
	if nntype == 'svm':
		svms = svm.SVC(gamma='scale',C=1,class_weight='balanced')
		return svms
	if nntype == 'knn':
		neigh = knn(n_neighbors=5)
		return neigh
	if nntype == 'autowave':
		model = Sequential()
		model.add(Conv1D(16, kernel_size=3, padding='same',input_shape=(128,68)))
		model.add(Activation('relu'))
		model.add(MaxPooling1D(pool_size=2, padding='same'))
		model.add(Dropout(0.7))

		model.add(Conv1D(32, kernel_size=3, padding='same'))
		model.add(Activation('relu'))
		model.add(MaxPooling1D(pool_size=2, padding='same'))
		model.add(Dropout(0.5))

		model.add(Conv1D(64, kernel_size=3, padding='same'))
		model.add(Activation('relu'))
		model.add(MaxPooling1D(pool_size=2, padding='same'))
		model.add(Dropout(0.5))

		model.add(Conv1D(64, kernel_size=3, padding='same'))
		model.add(Activation('relu'))
		model.add(UpSampling1D(size=2))
		model.add(Dropout(0.5))

		model.add(Conv1D(32, kernel_size=3, padding='same'))
		model.add(Activation('relu'))
		model.add(UpSampling1D(size=2))
		model.add(Dropout(0.5))

		model.add(Conv1D(32, kernel_size=3, padding='same'))
		model.add(Activation('relu'))
		model.add(UpSampling1D(size=2))
		model.add(Dropout(0.5))
		model.add(Conv1D(68, kernel_size=3, padding='same'))
		model.add(Activation('sigmoid'))

		return model

'''
#Get Kaggle Data
train,test,ytrain,ytest = getkaggle('Dog_1')
train = np.asarray(train)
test = np.asarray(test)
print(train.shape)
train = np.swapaxes(train,0,1)
test = np.swapaxes(test,0,1)
#train = np.swapaxes(train,1,2)
#test = np.swapaxes(test,1,2)
train = train.reshape(16,train.shape[1]*train.shape[2])
test = test.reshape(16,test.shape[1]*test.shape[2])
print(train.shape)
'''

'''Auto-Encoder for EEG Channels'''
'''
auto = Sequential()
auto.add(Dense(16,activation='relu',input_dim=16))
auto.add(Dense(16,activation='relu'))
auto.add(Dropout(0.5))
auto.add(Dense(16,activation='relu'))
auto.add(Dense(16,activation='relu'))
auto.add(Dropout(0.5))
auto.add(Dense(16,activation='sigmoid'))
#,validation_data=(x_test[0:(lentest//reduc)],y_test[0:(lentest//reduc)]),
adam = optimizers.adam(lr=0.001,amsgrad=True)
auto.compile(loss='mean_squared_error', optimizer=adam, metrics=['accuracy'])
auto.fit(train[:,0:3000].transpose(), train[:,0:3000].transpose(), 
          batch_size=32, epochs=10, verbose=1,validation_split=0.2,shuffle=True)
test_hist = auto.evaluate(test[:,0:3000].transpose(), test[:,0:3000].transpose(), verbose=1)
print(test_hist)


trains,tests,y_train20,y_test20 = segmentkag(train,test,ytrain,ytest,'dog',2)
trains = np.asarray(trains)
y_train20 = np.asarray(y_train20)
#print(trains.shape)

#print(y_train20.shape)
'''
'''Second: segment length'''
second = 3

pretime = 245760
np.set_printoptions(threshold=sys.maxsize)
'''768'''
'''Process Data'''
szstart2 = 579840
szend2 = 587520

signal, labels, szstart, szend = raw('EDMSE_pat_FR_1096_005.mat')
#die = IDT(signal[1,0:300])
#print(die.shape)
#hi = wavepack(signal[:,0:500])
#print(spectrograms(signal[:,0:700],256,64))
#IDT(signal[1,0:700])
#imf = EMDs(signal[1,0:3*400])
#labels[szstart2:szend2] = 1
#spectrograms(signal[:,0:5*256],256,64)

labels[szstart2 - pretime:szstart2] = 1
labels[szstart - pretime:szstart] = 1
xraw_train, y_train, xraw_test, y_test = segment_data(signal[:,0:szstart-15360],labels[0:szstart-15360],False,second)

#xraw_train1, y_train1, xraw_test1, y_test1 = segment_data(signal[:,szend:szstart2],labels[szend:szstart2],False,second)
xraw_train2, y_train2, xraw_test2, y_test2 = segment_data(signal[:,szend2+15360:],labels[szend2+15360:],False,second)

xraw_train += xraw_train2
y_train += y_train2
xraw_test += xraw_test2
y_test += y_test2


'''Get Ictal Signals from Other Files'''
for y in os.listdir('data'):
	if (y == '.DS_Store') or (y == 'EDMSE_pat_FR_1096_005.mat') or (y == '__MACOSX') or (y.find('EDMSE') == -1):
		continue
	path = os.path.join('data',y)
	signal1, labels1, szstart1, szend1 = raw(path)
	#print(labels1.shape)
	#print(signal1.shape)
	'''Get Preictal Signals'''
	if szstart1 > pretime:
		preictstart = szstart1 - pretime
	else:
		preictstart = 0
	labels1[preictstart:szstart1-15360] = 1
	xraw_train1, y_train1, xraw_test1, y_test1 = segment_data(signal1[:,0:szstart1-15360],labels1[0:szstart1-15360],False,second)
	if (szend1+15360) > len(signal1[1]):
		xraw_train2, y_train2, xraw_test2, y_test2 = segment_data(signal1[:,szend1:],labels1[szend1:],False,second)
	else:
		xraw_train2, y_train2, xraw_test2, y_test2 = segment_data(signal1[:,szend1+15360:],labels1[szend1+15360:],False,second)
	#print(y_train1)
	xraw_train += xraw_train1 + xraw_train2
	y_train += y_train1 + y_train2
	xraw_test += xraw_test1 + xraw_test2
	y_test += y_test1 + y_test2
	if y_train1 == []:
		os.remove(path)

'''Print Ratio of Ictal to Non-Ictal Training Data'''		
print(sum(y_train)/len(y_train))

x_train = []
x_test = []
#wavelet_raw(xraw_train[1])
'''
for x in tests:	
	feature = feature_extraction(x)
	x_train += [feature]

for t in trains:
	feature = feature_extraction(x)
	x_test += [feature]

'''
ah = 0
for x in xraw_train:
	#features = stfts(x,256,128)
	#features = spectrograms(x,1200,200)
	#features = wavelet_raw(x)
	features = feature_extraction(x)
	#features = IDTraw(x)
	if type(features) == int:
		pass
	else:
		print(features.shape)
		x_train += [features]
	'''
	accum = []
	for i in range(0,len(x)):
		accum += [LLE(x[i,:]),cordim(x[i,:]),dfas(x[i,:]),ApEn(x[i,:],2,2),skew(x[i,:]),kurtosis(x[i,:]),mean(x[i,:]),np.std(x[i,:])]
	x_train += [accum]
	'''
for x in xraw_test:
	#features = stfts(x,256,128)
	#features = spectrograms(x,1200,200)
	#features = wavelet_raw(x)
	features = feature_extraction(x)
	#features = IDTraw(x)
	if type(features) == int:
		pass
	else:
		print(features.shape)
		x_test += [features]
	'''
	accum = []
	for i in range(0,len(x)):
		accum += [LLE(x[i,:]),cordim(x[i,:]),dfas(x[i,:]),ApEN(x[i,:],2,2),skew(x[i,:]),kurtosis(x[i,:]),mean(x[i,:]),np.std(x[i,:])]
	x_test += [accum]
	'''
print('hi')

'''Standardize Input Data'''

'''Oversample'''
'''
for i in range(0,len(x_train),1):
	if y_train[i] == 1:
		for z in range(0,1):
			y_train += [1]
			x_train += [x_train[i]]


tempsx = []
tempsy = []
for i in range(0,len(x_train),1):
	if y_train[i] == 0:
		if randint(0,5) == 3:
			tempsx += [x_train[i]]
			tempsy += [0]
	else:
		tempsx += [x_train[i]]
		tempsy += [1]

x_train = tempsx
y_train = tempsy
'''

#x_train, y_train = shuffle(x_train,y_train)
#x_test, y_test = shuffle(x_test,y_test)
print(len(x_train))
x_train = np.asarray(x_train)
x_test = np.asarray(x_test)
np.save('x_train',x_train)
np.save('x_test',x_test)
#np.load(filename)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

shapetr = x_train.shape
shapete = x_test.shape
print(x_train.shape)
#print(y_train.shape)

#x_train, y_train = Adasyn(x_train, y_train)
#print('hi')
lentrain = len(x_train)
lentest = len(x_test)

#print(model.summary())
'''Reshape Input for FC'''
scalar = StandardScaler()

x_train = x_train.reshape(lentrain,shapetr[1]*shapetr[2])
x_test = x_test.reshape(lentest,shapetr[1]*shapetr[2])
print(x_train.shape)
x_train = np.nan_to_num(x_train)
x_test = np.nan_to_num(x_test)
print(x_train)
x_train = scalar.fit_transform(x_train)
x_test = scalar.fit_transform(x_test)

'''Apply Dimensionality Reduction'''
pca = PCA(n_components=200)
#ica = FastICA(algorithm='parallel',whiten=True,n_components=288, tol=0.01, max_iter=1000)
#lda = LDA(solver='svd')
#iso = Isomap(n_neighbors=5, n_components=288, eigen_solver='auto')
#ump = umap.UMAP(n_components=30)
#print('PCA Start')
x_train = pca.fit_transform(x_train)
x_test = pca.fit_transform(x_test)
#print('LDA start')
#x_train = lda.fit_transform(x_train,y_train20)
#x_test = lda.fit_transform(x_test,y_test20)
print(x_train.shape)

#print(x_train.flatten())
'''Standardize Input Data'''

x_train = scalar.fit_transform(x_train)
x_test = scalar.fit_transform(x_test)

'''Build Model'''
modeltype = 'svm'
class_weight = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
#print(class_weight)
model = Model(modeltype)

if modeltype == 'cnn':
	x_train = x_train.reshape(shapetr)
	x_test = x_test.reshape(shapete)

if (modeltype != 'svm') and (modeltype != 'knn'):
	adam = optimizers.adam(lr=0.001,amsgrad=True)
	model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
	print('THIS IS TRAIN LENGTH:')
	print(lentrain)
	reduc = 1
	#,validation_data=(x_test[0:(lentest//reduc)],y_test[0:(lentest//reduc)]),
	model.fit(x_train[0:(lentrain)], y_train[0:(lentrain)], 
	          batch_size=32, epochs=5, verbose=1,validation_split=0.2,shuffle=True,class_weight = {0:class_weight[0],1:class_weight[1]})
	test_hist = model.evaluate(x_test, y_test, verbose=1)
	y_pred = model.predict(x_test, verbose=0)
	#y_pred = y_pred.reshape(1,-1)
	
else:
	model.fit(x_train,y_train)
	y_pred = model.predict(x_test)
	#print(y_pred)

y_pred = y_pred.flatten()
#print(y_pred)
for i in range(0,len(y_pred),1):
	if y_pred[i] < 0.5:
		y_pred[i] = 0
	else:
		y_pred[i] = 1

#print(test_hist)

y_test = np.asarray(y_test)
#print('YTEST')
#print(y_test)
f1score = f1_score(y_test,y_pred)
confmat = confusion_matrix(y_test,y_pred, labels=[0,1])
#print(model.metrics_names)
print(f1score)
print(confmat)


'''x_trains = traindata, x_train = labels for LDA approximation'''
'''
winit = TruncatedNormal(mean=0,stddev=0.05)
binit = Constant(value=0)
approx = Sequential()
approx.add(Dense(2000,input_dim=8704,kernel_initializer=winit,
        bias_initializer=binit))
approx.add(Activation(activation='relu'))

approx.add(Dropout(0.7))
approx.add(Dense(500,kernel_initializer=winit,
        bias_initializer=binit))
approx.add(Activation(activation='relu'))

approx.add(Dropout(0.7))
approx.add(Dense(100,kernel_initializer=winit,
        bias_initializer=binit))
approx.add(Activation(activation='relu'))

approx.add(Dropout(0.5))
approx.add(Dense(1, activation='sigmoid',kernel_initializer=winit,
        bias_initializer=binit))
#approx.fit(x_trains,x_train)

approx.compile(loss='mean_squared_error', optimizer=adam, metrics=['accuracy'])
#
reduc = 1

approx.fit(x_trains[0:(lentrain//reduc)], x_train[0:(lentrain//reduc)], 
          batch_size=32, epochs=10, verbose=1,validation_split=0.2,validation_data=(x_tests[0:(lentest//reduc)],x_test[0:(lentest//reduc)]),shuffle=True)
y_pred = approx.predict(x_tests)
print(y_pred)
print(test_hist)
'''


/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
Using TensorFlow backend.


0.18418455465087588
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)
(24, 16)